# Loading libraries

In [6]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from time import sleep
import pandas as pd

# Get  the links from all team by season

In [6]:
def func_day(d):
    return d.split()[0]
def func_month(d):
    return d.split()[1]
def func_year(d):
    return d.split()[2]

dict_months = {
    "Janvier": "01",
    "Février": "02",
    "Mars": "03",
    "Avril": "04",
    "Mai": "05",
    "Juin": "06",
    "Juillet": "07",
    "Août": "08",
    "Septembre": "09",
    "Octobre": "10",
    "Novembre": "11",
    "Décembre": "12"
}


r = requests.get('https://www.fifaindex.com/fr/teams/')
soup = BeautifulSoup(r.text, 'html.parser')
fifas=soup.find("li",{"class":"breadcrumb-item dropdown"}).find_all("a")[1:]
lien_fifa=[]
for i in range(len(fifas)):
    lien_fifa.append("https://www.fifaindex.com"+fifas[i]["href"])
lien_fifa.pop(3)  #delete then fifa18wc (world  cup) 

def links_by_championnat(championnat):
    link_team_championanat=[]
    links_date =[]
    date_str=[]
    for i in range(len(lien_fifa)):  
        r = requests.get(lien_fifa[i])
        soup = BeautifulSoup(r.text, 'html.parser')    
        dates=soup.find_all("li",{"class":"breadcrumb-item dropdown"})[1].find_all("a")[1:]

        for j in range(len(dates)):
            links_date.append("https://www.fifaindex.com"+dates[j]["href"]+championnat)
            date_str.append(dates[j].text)
    df=pd.DataFrame(list(zip(date_str, links_date)), date_str,columns=['dates',"links_date"])
    df['test']=df['dates'].map(func_day)+'-'+df['dates'].map(func_month).map(dict_months)+'-'+df['dates'].map(func_year)
    df['test'] = pd.to_datetime(df['test'],format="%d-%m-%Y" )
    df["month"]=df["test"].dt.month
    df["year"]=df["test"].dt.year
    df.sort_values(by=["year","month"], ascending=False)
    df.drop_duplicates(subset = ["year","month"], keep = 'first', inplace=True)
    links_date=df.links_date.values.tolist()       
    
    for i in range(len(links_date)):  
        r = requests.get(links_date[i])
        soup = BeautifulSoup(r.text, 'html.parser')
        Teams=soup.find("div",{"class":"responsive-table table-rounded"}).find_all("a",{"class":"link-team"})
        for j in range(len(Teams)):
            if j%2:
                link_team_championanat.append("https://www.fifaindex.com"+Teams[j]["href"])
        sleep(5)
    return link_team_championanat

In [7]:
filter_serieA = "?league=31&order=desc"
filter_bundesligua = "?league=19&order=desc"
filter_ligue1 = "?league=16&order=desc"
filter_premiere_league = "?league=13&order=desc"
filter_liga = "?league=53&order=desc"

In [8]:
links_date_serieA=links_by_championnat(filter_serieA)
df_links_date_serieA=pd.DataFrame({"links_date_serieA": links_date_serieA})
df_links_date_serieA.to_csv('input/df_links_date_serieA.csv',sep=";")

In [9]:
links_date_bundesligua=links_by_championnat(filter_bundesligua )
df_links_date_bundesligua=pd.DataFrame({"links_date_bundesligua": links_date_bundesligua})
df_links_date_bundesligua.to_csv('input/df_links_date_bundesligua.csv',sep=";")

In [11]:
links_date_premiere_league=links_by_championnat(filter_premiere_league )
df_links_date_premiere_league=pd.DataFrame({"links_date_premiere_league": links_date_premiere_league})
df_links_date_premiere_league.to_csv('input/df_links_date_premiere_league.csv',sep=";")

In [12]:
links_date_liga=links_by_championnat(filter_liga )
df_links_date_liga=pd.DataFrame({"links_date_liga": links_date_liga})
df_links_date_liga.to_csv('input/df_links_date_liga.csv',sep=";")

In [13]:
links_date_ligue1=links_by_championnat(filter_ligue1)
df_links_date_ligue1=pd.DataFrame({"links_date_ligue1": links_date_ligue1})
df_links_date_ligue1.to_csv('input/df_links_date_ligue1.csv',sep=";")

# Dataframe TEAMS

In [14]:
def dataframe_by_championnat(links_date_championnat): 
    df_teams_championnat= pd.DataFrame(columns=['team','date','attaque','milieu','defense','budget_transfert'])
    for link in range(len(links_date_championnat)):
        r = requests.get(links_date_championnat[link])
        soup = BeautifulSoup(r.text, 'html.parser')
        records = []
        team=soup.find_all("div",{"class":"col-sm-6 col-md-7"})[0].find("h1").text
        date=soup.find_all("li",{"class":"breadcrumb-item dropdown"})[1].find_all("a")[0].text
        attaque=soup.find_all("li",{"class": "list-group-item"})[1].find("span").text
        milieu=soup.find_all("li",{"class": "list-group-item"})[2].find("span").text
        defense=soup.find_all("li",{"class": "list-group-item"})[3].find("span").text
        try : 
            budget_transfert=soup.find_all("li",{"class": "list-group-item"})[4].find("span",{"class":"data-currency data-currency-euro float-right"}).text
        except :
            budget_transfert=''
        records.append((team,date, attaque, milieu,defense,budget_transfert))
        df_team = pd.DataFrame(records, columns=['team','date','attaque','milieu','defense','budget_transfert'])
        df_teams_championnat=df_teams_championnat.append(df_team)
        sleep(1)
    return df_teams_championnat

In [ ]:
df_links_date_serieA = pd.read_csv('input/df_links_date_serieA.csv',sep=";")
links_date_serieA=df_links_date_serieA.links_date_serieA.values.tolist()

df_teams_serieA=dataframe_by_championnat(links_date_serieA)
df_teams_serieA.to_csv('input/df_teams_serieA.csv',sep=";")

In [ ]:
df_links_date_bundesligua = pd.read_csv('input/df_links_date_bundesligua.csv',sep=";")
links_date_bundesligua=df_links_date_bundesligua.links_date_bundesligua.values.tolist()

df_teams_budesligua=dataframe_by_championnat(links_date_bundesligua)
df_teams_budesligua.to_csv('input/df_teams_budesligua.csv',sep=";")

In [ ]:
df_links_date_premiere_league = pd.read_csv('input/df_links_date_premiere_league.csv',sep=";")
links_date_premiere_league=df_links_date_premiere_league.links_date_premiere_league.values.tolist()

df_teams_premiere_league=dataframe_by_championnat(links_date_premiere_league)
df_teams_premiere_league.to_csv('input/df_teams_premiere_league.csv',sep=";")

In [ ]:
df_links_date_liga = pd.read_csv('input/df_links_date_liga.csv',sep=";")
links_date_liga=df_links_date_liga.links_date_liga.values.tolist()

df_teams_liga=dataframe_by_championnat(links_date_liga)
df_teams_liga.to_csv('input/df_teams_liga.csv',sep=";")

In [ ]:
df_links_date_ligue1 = pd.read_csv('input/df_links_date_ligue1.csv',sep=";")
links_date_ligue1=df_links_date_ligue1.links_date_ligue1.values.tolist()

df_teams_ligue1=dataframe_by_championnat(links_date_ligue1)
df_teams_ligue1.to_csv('input/df_teams_ligue1.csv',sep=";")

In [ ]:
df_teams_ligue1= pd.read_csv("input/df_teams_ligue1.csv",sep=";",low_memory=False)
df_teams_liga= pd.read_csv("input/df_teams_liga.csv",sep=";",low_memory=False)
df_teams_premiere_league= pd.read_csv("input/df_teams_premiere_league.csv",sep=";",low_memory=False)
df_teams_budesligua= pd.read_csv("input/df_teams_budesligua.csv",sep=";",low_memory=False)
df_teams_serieA= pd.read_csv("input/df_teams_serieA.csv",sep=";",low_memory=False)
df_teams=pd.concat([df_teams_ligue1, df_teams_liga,df_teams_premiere_league,df_teams_budesligua,df_teams_serieA], ignore_index=True)
df_teams.to_csv('intermed/df_teams.csv', sep=";")   

# Dataframe PLAYERS 

In [20]:
def links_players_by_championnat(links_date_championnat) :
    links_player=[]
    for i in range(len(links_date_championnat)):  
        r = requests.get(links_date_championnat[i])
        soup = BeautifulSoup(r.text, 'html.parser')
        players=soup.find("div",{"class":"responsive-table table-rounded"}).find_all("tr")[1:]
        for j in range(len(players)):
            links_player.append("https://www.fifaindex.com"+players[j].find_all("a")[0]["href"])
        sleep(1)
    return links_player

In [21]:
df_links_date_serieA = pd.read_pickle('input/df_links_date_serieA.pkl')
links_date_serieA=df_links_date_serieA.links_date_serieA.values.tolist()

links_player_serieA=links_players_by_championnat(links_date_serieA)
df_links_player_serieA=pd.DataFrame({"links_player_serieA": links_player_serieA})
df_links_player_serieA.to_pickle('input/df_links_player_serieA.pkl')

In [22]:
df_links_date_bundesligua = pd.read_pickle('input/df_links_date_bundesligua.pkl')
links_date_bundesligua=df_links_date_bundesligua.links_date_bundesligua.values.tolist()

links_player_bunedesligua=links_players_by_championnat(links_date_bundesligua )
df_links_player_bunedesligua=pd.DataFrame({"links_player_bunedesligua": links_player_bunedesligua})
df_links_player_bunedesligua.to_pickle('input/df_links_player_bunedesligua.pkl')

In [23]:
df_links_date_premiere_league = pd.read_pickle('input/df_links_date_premiere_league.pkl')
links_date_premiere_league=df_links_date_premiere_league.links_date_premiere_league.values.tolist()

links_player_premiere_league=links_players_by_championnat(links_date_premiere_league)
df_links_player_premiere_league=pd.DataFrame({"links_player_premiere_league": links_player_premiere_league})
df_links_player_premiere_league.to_pickle('input/df_links_player_premiere_league.pkl')

In [24]:
df_links_date_liga = pd.read_pickle('input/df_links_date_liga.pkl')
links_date_liga=df_links_date_liga.links_date_liga.values.tolist()

links_player_liga=links_players_by_championnat(links_date_liga )
df_links_player_liga=pd.DataFrame({"links_player_liga": links_player_liga})
df_links_player_liga.to_pickle('input/df_links_player_liga.pkl')

In [25]:
df_links_date_ligue1 = pd.read_pickle('input/df_links_date_ligue1.pkl')
links_date_ligue1=df_links_date_ligue1.links_date_ligue1.values.tolist()

links_player_ligue1=links_players_by_championnat(links_date_ligue1 )
df_links_player_ligue1=pd.DataFrame({"links_player_ligue1": links_player_ligue1})
df_links_player_ligue1.to_pickle('input/df_links_player_ligue1.pkl')

In [10]:
def dataframe_players_by_champinonat(links_players):
    df_players=pd.DataFrame(columns=['player','club','date','taille','poid','meilleur_pied','age','valeur','salaire','controles',
                         'dribbles','marquage','tacle_glisse','tacle_debout','engagement','reactivite','placement',
                         'intercept','vista','discipline','passe_centre','passe_courte','passe_longue','phy_acceleration',
                         'phy_endurance','phy_force',' phy_equilibre','phy_vitesse','phy_agilite','phy_detente','tir_tete',
                         'tir_frappe','tir_finition','tir_loin','tir_effet','tir_precision','tir_penalty','tir_volee',
                         'gardien_placement','gardien_plongeon','gardien_main','gardien_pied','gardien_reflexe'])


    for i in range(len(links_players)):
        r = requests.get(links_players[i])
        soup = BeautifulSoup(r.text, 'html.parser')
        records = []
        team=soup.find_all("div",{"class":"col-12 col-sm-6 col-lg-6 team"})
        info=soup.find_all("div",{"class":"col-sm-6"})
        cards=soup.find_all("div",{"class":"col-12 col-md-4 item"})

        
        try:
            try:
                try :
                    player=info[1].contents[1].find_all("h5",{"class":"card-header"})[0].contents[0]
                except :
                    player=""
                try:
                    c=team[1].find("div",{"class":"card-body"}).find('button').text
                    club=team[1].find("h5").find_all("a")[-1].text
                except:
                    c=team[0].find("div",{"class":"card-body"}).find('button').text
                    club=team[0].find("h5").find_all("a")[-1].text
                date=soup.find_all("li",{"class":"breadcrumb-item dropdown"})[1].find_all("a")[0].text
                taille=info[1].find_all("span",{"class":"data-units data-units-metric"})[0].text
                poid=info[1].find_all("span",{"class":"data-units data-units-metric"})[1].text
                meilleur_pied=info[1].find_all("p")[2].find("span").text
                age=info[1].find_all("p")[4].find("span").text
                try : 
                    valeur=info[1].find_all("p",{"class":"data-currency data-currency-euro"})[0].find("span").text
                    salaire=info[1].find_all("p",{"class":"data-currency data-currency-euro"})[1].find("span").text
                except :
                    valeur=""
                    salaire=""

                controles=cards[0].contents[1].find_all("p")[0].contents[1].text
                dribbles= cards[0].contents[1].find_all("p")[1].contents[1].text

                marquage=cards[1].contents[1].find_all("p")[0].contents[1].text
                tacle_glisse=cards[1].contents[1].find_all("p")[1].contents[1].text
                tacle_debout=cards[1].contents[1].find_all("p")[2].contents[1].text

                engagement = cards[2].contents[1].find_all("p")[0].contents[1].text
                reactivite = cards[2].contents[1].find_all("p")[1].contents[1].text
                placement = cards[2].contents[1].find_all("p")[2].contents[1].text
                intercept = cards[2].contents[1].find_all("p")[3].contents[1].text
                vista = cards[2].contents[1].find_all("p")[4].contents[1].text
                try :
                    discipline = cards[2].contents[1].find_all("p")[5].contents[1].text
                except :
                    discipline=""
                passe_centre=cards[3].contents[1].find_all("p")[0].contents[1].text
                passe_courte=cards[3].contents[1].find_all("p")[1].contents[1].text
                passe_longue=cards[3].contents[1].find_all("p")[2].contents[1].text

                phy_acceleration=cards[4].contents[1].find_all("p")[0].contents[1].text
                phy_endurance=cards[4].contents[1].find_all("p")[1].contents[1].text
                phy_force=cards[4].contents[1].find_all("p")[2].contents[1].text
                phy_equilibre=cards[4].contents[1].find_all("p")[3].contents[1].text
                phy_vitesse=cards[4].contents[1].find_all("p")[4].contents[1].text
                phy_agilite=cards[4].contents[1].find_all("p")[5].contents[1].text
                phy_detente=cards[4].contents[1].find_all("p")[6].contents[1].text

                tir_tete=cards[5].contents[1].find_all("p")[0].contents[1].text
                tir_frappe=cards[5].contents[1].find_all("p")[1].contents[1].text
                tir_finition=cards[5].contents[1].find_all("p")[2].contents[1].text
                tir_loin=cards[5].contents[1].find_all("p")[3].contents[1].text
                tir_effet=cards[5].contents[1].find_all("p")[4].contents[1].text
                tir_precision=cards[5].contents[1].find_all("p")[5].contents[1].text
                tir_penalty=cards[5].contents[1].find_all("p")[6].contents[1].text
                tir_volee=cards[5].contents[1].find_all("p")[7].contents[1].text

                gardien_placement=cards[6].contents[1].find_all("p")[0].contents[1].text
                gardien_plongeon=cards[6].contents[1].find_all("p")[1].contents[1].text
                gardien_main=cards[6].contents[1].find_all("p")[2].contents[1].text
                gardien_pied=cards[6].contents[1].find_all("p")[3].contents[1].text
                gardien_reflexe=cards[6].contents[1].find_all("p")[4].contents[1].text
            except :
                    try:
                        try :
                            player=info[1].contents[1].find_all("h5",{"class":"card-header"})[0].contents[0]
                        except :
                            player=""
                        try:
                            c=team[1].find("div",{"class":"card-body"}).find('button').text
                            club=team[1].find("h5").find_all("a")[-1].text
                        except:
                            c=team[0].find("div",{"class":"card-body"}).find('button').text
                            club=team[0].find("h5").find_all("a")[-1].text
                        date=soup.find_all("li",{"class":"breadcrumb-item dropdown"})[1].find_all("a")[0].text
                        taille=info[1].find_all("span",{"class":"data-units data-units-metric"})[0].text
                        poid=info[1].find_all("span",{"class":"data-units data-units-metric"})[1].text
                        meilleur_pied=info[1].find_all("p")[2].find("span").text
                        age=info[1].find_all("p")[4].find("span").text
                        try : 
                            valeur=info[1].find_all("p",{"class":"data-currency data-currency-euro"})[0].find("span").text
                            salaire=info[1].find_all("p",{"class":"data-currency data-currency-euro"})[1].find("span").text
                        except :
                            valeur=""
                            salaire=""

                        controles=cards[0].contents[1].find_all("p")[0].contents[-1].text
                        dribbles= cards[0].contents[1].find_all("p")[1].contents[-1].text

                        marquage=cards[1].contents[1].find_all("p")[0].contents[-1].text
                        tacle_glisse=cards[1].contents[1].find_all("p")[1].contents[-1].text
                        tacle_debout=""

                        engagement = cards[2].contents[1].find_all("p")[0].contents[-1].text
                        reactivite = cards[2].contents[1].find_all("p")[2].contents[-1].text
                        placement = ""
                        intercept = ""
                        vista =""
                        try :
                            discipline = cards[2].contents[1].find_all("p")[1].contents[-1].text
                        except :
                            discipline=""
                        passe_centre=cards[3].contents[1].find_all("p")[0].contents[-1].text
                        passe_courte=cards[3].contents[1].find_all("p")[1].contents[-1].text
                        passe_longue=cards[3].contents[1].find_all("p")[2].contents[-1].text

                        phy_acceleration=cards[4].contents[1].find_all("p")[0].contents[-1].text
                        phy_endurance=cards[4].contents[1].find_all("p")[1].contents[-1].text
                        phy_force=cards[4].contents[1].find_all("p")[2].contents[-1].text
                        phy_equilibre=""
                        phy_vitesse=cards[4].contents[1].find_all("p")[3].contents[-1].text
                        phy_agilite=""
                        phy_detente=""

                        tir_tete=cards[5].contents[1].find_all("p")[0].contents[-1].text
                        tir_frappe=cards[5].contents[1].find_all("p")[1].contents[-1].text
                        tir_finition=cards[5].contents[1].find_all("p")[2].contents[-1].text
                        tir_loin=cards[5].contents[1].find_all("p")[3].contents[-1].text
                        tir_effet=""
                        tir_precision=cards[5].contents[1].find_all("p")[4].contents[-1].text
                        tir_penalty=""
                        tir_volee=""

                        gardien_placement=cards[6].contents[1].find_all("p")[2].contents[-1].text
                        gardien_plongeon=cards[6].contents[1].find_all("p")[3].contents[-1].text
                        gardien_main=cards[6].contents[1].find_all("p")[1].contents[-1].text
                        gardien_pied=""
                        gardien_reflexe=cards[6].contents[1].find_all("p")[0].contents[-1].text
                    except : 
                        try :
                            player=info[1].contents[1].find_all("h5",{"class":"card-header"})[0].contents[0]
                        except :
                            player=""
                        try:
                            c=team[1].find("div",{"class":"card mb-5"}).find('button').text
                            club=team[1].find("h5").find_all("a")[-1].text
                        except:
                            c=team[0].find("div",{"class":"card mb-5"}).find('button').text
                            club=team[0].find("h5").find_all("a")[-1].text
                        date=soup.find_all("li",{"class":"breadcrumb-item dropdown"})[1].find_all("a")[0].text
                        taille=info[1].find_all("span",{"class":"data-units data-units-metric"})[0].text
                        poid=info[1].find_all("span",{"class":"data-units data-units-metric"})[1].text
                        meilleur_pied=info[1].find_all("p")[2].find("span").text
                        age=info[1].find_all("p")[4].find("span").text
                        try : 
                            valeur=info[1].find_all("p",{"class":"data-currency data-currency-euro"})[0].find("span").text
                            salaire=info[1].find_all("p",{"class":"data-currency data-currency-euro"})[1].find("span").text
                        except :
                            valeur=""
                            salaire=""

                        controles=cards[0].contents[1].find_all("p")[0].contents[-1].text
                        dribbles= cards[0].contents[1].find_all("p")[1].contents[-1].text

                        marquage=cards[1].contents[1].find_all("p")[0].contents[-1].text
                        tacle_glisse=cards[1].contents[1].find_all("p")[1].contents[-1].text
                        tacle_debout=""

                        engagement = cards[2].contents[1].find_all("p")[0].contents[-1].text
                        reactivite = cards[2].contents[1].find_all("p")[1].contents[-1].text
                        placement = ""
                        intercept = ""
                        vista =""

                        discipline = cards[2].contents[1].find_all("p")[2].contents[-1].text

                        passe_centre=cards[3].contents[1].find_all("p")[0].contents[-1].text
                        passe_courte=cards[3].contents[1].find_all("p")[1].contents[-1].text
                        passe_longue=cards[3].contents[1].find_all("p")[2].contents[-1].text

                        phy_acceleration=cards[4].contents[1].find_all("p")[0].contents[-1].text
                        phy_endurance=cards[4].contents[1].find_all("p")[2].contents[-1].text
                        phy_force=cards[4].contents[1].find_all("p")[3].contents[-1].text
                        phy_equilibre=cards[4].contents[1].find_all("p")[4].contents[-1].text
                        phy_vitesse=cards[4].contents[1].find_all("p")[1].contents[-1].text
                        phy_agilite=""
                        phy_detente=""

                        tir_tete=cards[5].contents[1].find_all("p")[0].contents[-1].text
                        tir_frappe=cards[5].contents[1].find_all("p")[2].contents[-1].text
                        tir_finition=""
                        try: 
                            tir_loin=cards[5].contents[1].find_all("p")[3].contents[-1].text
                        except : 
                            tir_loin=""
                        tir_effet=""
                        tir_precision=cards[5].contents[1].find_all("p")[1].contents[-1].text
                        tir_penalty=""
                        tir_volee=""

                        gardien_placement=cards[6].contents[1].find_all("p")[3].contents[-1].text
                        gardien_plongeon=""
                        gardien_main=cards[6].contents[1].find_all("p")[2].contents[-1].text
                        gardien_pied=""
                        gardien_reflexe=cards[6].contents[1].find_all("p")[0].contents[-1].text    
            records.append((player,club,date,taille,poid,meilleur_pied,age,valeur,
                                      salaire,controles,dribbles,marquage,tacle_glisse,tacle_debout,engagement,
                                      reactivite,placement,intercept,vista,discipline,passe_centre,passe_courte,
                                      passe_longue,phy_acceleration,phy_endurance,phy_force,phy_equilibre,phy_vitesse,
                                      phy_agilite,phy_detente,tir_tete,tir_frappe,tir_finition,tir_loin,tir_effet,
                                      tir_precision,tir_penalty,tir_volee,gardien_placement,gardien_plongeon,gardien_main,
                                      gardien_pied,gardien_reflexe))
            df_player = pd.DataFrame(records, columns=['player','club','date','taille','poid','meilleur_pied','age','valeur',
                                      'salaire','controles','dribbles','marquage','tacle_glisse','tacle_debout','engagement',
                                      'reactivite','placement','intercept','vista','discipline','passe_centre','passe_courte',
                                      'passe_longue','phy_acceleration','phy_endurance','phy_force','phy_equilibre','phy_vitesse',
                                      'phy_agilite','phy_detente','tir_tete','tir_frappe','tir_finition','tir_loin','tir_effet',
                                      'tir_precision','tir_penalty','tir_volee','gardien_placement','gardien_plongeon','gardien_main',
                                      'gardien_pied','gardien_reflexe'])

            df_players=df_players.append(df_player)
            sleep(1)
        except:
            print(links_players[i])
    return df_players

In [ ]:
def create_chunks(list_name,n):
    for i in range(0,len(list_name),n):
        yield list_name[i:i+n]

df_links_player_bundesligua = pd.read_pickle('input/df_links_player_bunedesligua.pkl')
links_player_bundesligua=df_links_player_bundesligua.links_player_bunedesligua.values.tolist()
list_players=list(create_chunks(links_player_bundesligua,100))

for element in range(len(list_players)):
    if element ==0:
        df_players_bundesligua = pd.DataFrame(columns=['player','club','date','taille','poid','meilleur_pied','age','valeur','salaire','controles',
                    'dribbles','marquage','tacle_glisse','tacle_debout','engagement','reactivite','placement',
                    'intercept','vista','discipline','passe_centre','passe_courte','passe_longue','phy_acceleration',
                    'phy_endurance','phy_force',' phy_equilibre','phy_vitesse','phy_agilite','phy_detente','tir_tete',
                    'tir_frappe','tir_finition','tir_loin','tir_effet','tir_precision','tir_penalty','tir_volee',
                    'gardien_placement','gardien_plongeon','gardien_main','gardien_pied','gardien_reflexe'])
        df_players_bundesligua.to_csv('input/df_players_bundesligua.csv', sep=";")  
        
    df=dataframe_players_by_champinonat(list_players[element])
    df_players_bundesligua=pd.read_csv('input/df_players_bundesligua.csv', sep=";")    
    df_players_bundesligua=df_players_bundesligua[['player','club','date','taille','poid','meilleur_pied','age','valeur','salaire','controles',
                         'dribbles','marquage','tacle_glisse','tacle_debout','engagement','reactivite','placement',
                         'intercept','vista','discipline','passe_centre','passe_courte','passe_longue','phy_acceleration',
                         'phy_endurance','phy_force',' phy_equilibre','phy_vitesse','phy_agilite','phy_detente','tir_tete',
                         'tir_frappe','tir_finition','tir_loin','tir_effet','tir_precision','tir_penalty','tir_volee',
                         'gardien_placement','gardien_plongeon','gardien_main','gardien_pied','gardien_reflexe']]
    df_players_bundesligua=df_players_bundesligua.append(df)
    df_players_bundesligua.to_csv('input/df_players_bundesligua.csv', sep=";")  

In [ ]:
def create_chunks(list_name,n):
    for i in range(0,len(list_name),n):
        yield list_name[i:i+n]


df_links_player_ligue1 = pd.read_pickle('input/df_links_player_ligue1.pkl')
links_player_ligue1=df_links_player_ligue1.links_player_ligue1.values.tolist()

list_players=list(create_chunks(links_player_ligue1,100))

for element in range(len(list_players)):
    if element ==0:
        df_players_ligue1=pd.DataFrame(columns=['player','club','date','taille','poid','meilleur_pied','age','valeur','salaire','controles',
                         'dribbles','marquage','tacle_glisse','tacle_debout','engagement','reactivite','placement',
                         'intercept','vista','discipline','passe_centre','passe_courte','passe_longue','phy_acceleration',
                         'phy_endurance','phy_force',' phy_equilibre','phy_vitesse','phy_agilite','phy_detente','tir_tete',
                         'tir_frappe','tir_finition','tir_loin','tir_effet','tir_precision','tir_penalty','tir_volee',
                         'gardien_placement','gardien_plongeon','gardien_main','gardien_pied','gardien_reflexe'])
        df_players_ligue1.to_csv('input/df_players_ligue1.csv', sep=";")  
        
    df=dataframe_players_by_champinonat(list_players[element])
    df_players_ligue1=pd.read_csv('input/df_players_ligue1.csv', sep=";")    
    df_players_ligue1=df_players_ligue1[['player','club','date','taille','poid','meilleur_pied','age','valeur','salaire','controles',
                         'dribbles','marquage','tacle_glisse','tacle_debout','engagement','reactivite','placement',
                         'intercept','vista','discipline','passe_centre','passe_courte','passe_longue','phy_acceleration',
                         'phy_endurance','phy_force',' phy_equilibre','phy_vitesse','phy_agilite','phy_detente','tir_tete',
                         'tir_frappe','tir_finition','tir_loin','tir_effet','tir_precision','tir_penalty','tir_volee',
                         'gardien_placement','gardien_plongeon','gardien_main','gardien_pied','gardien_reflexe']]
    df_players_ligue1=df_players_ligue1.append(df)
    df_players_ligue1.to_csv('input/df_players_ligue1.csv', sep=";")   

In [ ]:
def create_chunks(list_name,n):
    for i in range(0,len(list_name),n):
        yield list_name[i:i+n]


df_links_player_liga= pd.read_pickle('input/df_links_player_liga.pkl')
links_player_liga=df_links_player_liga.links_player_liga.values.tolist()

list_players=list(create_chunks(links_player_liga,100))

for element in range(len(list_players)):
    if element ==0:
        df_players_liga=pd.DataFrame(columns=['player','club','date','taille','poid','meilleur_pied','age','valeur','salaire','controles',
                         'dribbles','marquage','tacle_glisse','tacle_debout','engagement','reactivite','placement',
                         'intercept','vista','discipline','passe_centre','passe_courte','passe_longue','phy_acceleration',
                         'phy_endurance','phy_force',' phy_equilibre','phy_vitesse','phy_agilite','phy_detente','tir_tete',
                         'tir_frappe','tir_finition','tir_loin','tir_effet','tir_precision','tir_penalty','tir_volee',
                         'gardien_placement','gardien_plongeon','gardien_main','gardien_pied','gardien_reflexe'])
        df_players_liga.to_csv('input/df_players_liga.csv', sep=";")  
        
    df=dataframe_players_by_champinonat(list_players[element])
    df_players_liga=pd.read_csv('input/df_players_liga.csv', sep=";")    
    df_players_liga=df_players_liga[['player','club','date','taille','poid','meilleur_pied','age','valeur','salaire','controles',
                         'dribbles','marquage','tacle_glisse','tacle_debout','engagement','reactivite','placement',
                         'intercept','vista','discipline','passe_centre','passe_courte','passe_longue','phy_acceleration',
                         'phy_endurance','phy_force',' phy_equilibre','phy_vitesse','phy_agilite','phy_detente','tir_tete',
                         'tir_frappe','tir_finition','tir_loin','tir_effet','tir_precision','tir_penalty','tir_volee',
                         'gardien_placement','gardien_plongeon','gardien_main','gardien_pied','gardien_reflexe']]
    df_players_liga=df_players_liga.append(df)
    df_players_liga.to_csv('input/df_players_liga.csv', sep=";")   

In [ ]:
def create_chunks(list_name,n):
    for i in range(0,len(list_name),n):
        yield list_name[i:i+n]

df_links_player_premiere_league = pd.read_pickle('input/df_links_player_premiere_league.pkl')
links_player_premiere_league=df_links_player_premiere_league.links_player_premiere_league.values.tolist()

list_players=list(create_chunks(links_player_premiere_league,100))

for element in range(len(list_players)):
    if element ==0:
        df_players_premiere_league=pd.DataFrame(columns=['player','club','date','taille','poid','meilleur_pied','age','valeur','salaire','controles',
                         'dribbles','marquage','tacle_glisse','tacle_debout','engagement','reactivite','placement',
                         'intercept','vista','discipline','passe_centre','passe_courte','passe_longue','phy_acceleration',
                         'phy_endurance','phy_force',' phy_equilibre','phy_vitesse','phy_agilite','phy_detente','tir_tete',
                         'tir_frappe','tir_finition','tir_loin','tir_effet','tir_precision','tir_penalty','tir_volee',
                         'gardien_placement','gardien_plongeon','gardien_main','gardien_pied','gardien_reflexe'])
        df_players_premiere_league.to_csv('input/df_players_premiere_league.csv', sep=";")  
        
    df=dataframe_players_by_champinonat(list_players[element])
    df_players_premiere_league=pd.read_csv('input/df_players_premiere_league.csv', sep=";")    
    df_players_premiere_league=df_players_premiere_league[['player','club','date','taille','poid','meilleur_pied','age','valeur','salaire','controles',
                         'dribbles','marquage','tacle_glisse','tacle_debout','engagement','reactivite','placement',
                         'intercept','vista','discipline','passe_centre','passe_courte','passe_longue','phy_acceleration',
                         'phy_endurance','phy_force',' phy_equilibre','phy_vitesse','phy_agilite','phy_detente','tir_tete',
                         'tir_frappe','tir_finition','tir_loin','tir_effet','tir_precision','tir_penalty','tir_volee',
                         'gardien_placement','gardien_plongeon','gardien_main','gardien_pied','gardien_reflexe']]
    df_players_premiere_league=df_players_premiere_league.append(df)
    df_players_premiere_league.to_csv('input/df_players_premiere_league.csv', sep=";")   

In [ ]:
def create_chunks(list_name,n):
    for i in range(0,len(list_name),n):
        yield list_name[i:i+n]


df_links_player_serieA = pd.read_pickle('input/df_links_player_serieA.pkl')
links_player_serieA=df_links_player_serieA.links_player_serieA.values.tolist()
list_players=list(create_chunks(links_player_serieA,100))

for element in range(len(list_players)):
    if element ==0:
        df_players_serieA = pd.DataFrame(columns=['player','club','date','taille','poid','meilleur_pied','age','valeur','salaire','controles',
                    'dribbles','marquage','tacle_glisse','tacle_debout','engagement','reactivite','placement',
                    'intercept','vista','discipline','passe_centre','passe_courte','passe_longue','phy_acceleration',
                    'phy_endurance','phy_force',' phy_equilibre','phy_vitesse','phy_agilite','phy_detente','tir_tete',
                    'tir_frappe','tir_finition','tir_loin','tir_effet','tir_precision','tir_penalty','tir_volee',
                    'gardien_placement','gardien_plongeon','gardien_main','gardien_pied','gardien_reflexe'])
        df_players_serieA.to_csv('input/df_players_serieA.csv', sep=";")  
        
    df=dataframe_players_by_champinonat(list_players[element])
    df_players_serieA=pd.read_csv('input/df_players_bundesligua.csv', sep=";")    
    df_players_serieA=df_players_serieA[['player','club','date','taille','poid','meilleur_pied','age','valeur','salaire','controles',
                         'dribbles','marquage','tacle_glisse','tacle_debout','engagement','reactivite','placement',
                         'intercept','vista','discipline','passe_centre','passe_courte','passe_longue','phy_acceleration',
                         'phy_endurance','phy_force',' phy_equilibre','phy_vitesse','phy_agilite','phy_detente','tir_tete',
                         'tir_frappe','tir_finition','tir_loin','tir_effet','tir_precision','tir_penalty','tir_volee',
                         'gardien_placement','gardien_plongeon','gardien_main','gardien_pied','gardien_reflexe']]
    df_players_serieA=df_players_serieA.append(df)
    df_players_serieA.to_csv('input/df_players_serieA.csv', sep=";")   

In [ ]:
def create_chunks(list_name,n):
    for i in range(0,len(list_name),n):
        yield list_name[i:i+n]


df_links_player_serieA = pd.read_pickle('input/df_links_player_serieA.pkl')
links_player_serieA=df_links_player_serieA.links_player_serieA.values.tolist()
list_players=list(create_chunks(links_player_serieA,100))

for element in range(len(list_players)):
    if element ==0:
        df_players_serieA=pd.DataFrame(columns=['player','club','date','taille','poid','meilleur_pied','age','valeur','salaire','controles',
                         'dribbles','marquage','tacle_glisse','tacle_debout','engagement','reactivite','placement',
                         'intercept','vista','discipline','passe_centre','passe_courte','passe_longue','phy_acceleration',
                         'phy_endurance','phy_force',' phy_equilibre','phy_vitesse','phy_agilite','phy_detente','twir_tete',
                         'tir_frappe','tir_finition','tir_loin','tir_effet','tir_precision','tir_penalty','tir_volee',
                         'gardien_placement','gardien_plongeon','gardien_main','gardien_pied','gardien_reflexe'])
        df_players_serieA.to_csv('input/df_players_serieA.csv', sep=";")  
        
    df=dataframe_players_by_champinonat(list_players[element])
    df_players_serieA=pd.read_csv('input/df_players_serieA.csv', sep=";")    
    df_players_serieA=df_players_serieA[['player','club','date','taille','poid','meilleur_pied','age','valeur','salaire','controles',
                         'dribbles','marquage','tacle_glisse','tacle_debout','engagement','reactivite','placement',
                         'intercept','vista','discipline','passe_centre','passe_courte','passe_longue','phy_acceleration',
                         'phy_endurance','phy_force',' phy_equilibre','phy_vitesse','phy_agilite','phy_detente','tir_tete',
                         'tir_frappe','tir_finition','tir_loin','tir_effet','tir_precision','tir_penalty','tir_volee',
                         'gardien_placement','gardien_plongeon','gardien_main','gardien_pied','gardien_reflexe']]
    df_players_serieA=df_players_serieA.append(df)
    df_players_serieA.to_csv('input/df_players_serieA.csv', sep=";")  

In [ ]:
df_players_ligue1= pd.read_csv("input/df_players_ligue1.csv",sep=";",low_memory=False)
df_players_bundesligua= pd.read_csv("input/df_players_bundesligua.csv",sep=";",low_memory=False)
df_players_premiere_league= pd.read_csv("input/df_players_premiere_league.csv",sep=";",low_memory=False)
df_players_serieA= pd.read_csv("input/df_players_serieA.csv",sep=";",low_memory=False)
df_players_liga= pd.read_csv("input/df_players_liga.csv",sep=";",low_memory=False)
df_liste_qui_ne_fonctionne_pas2= pd.read_csv("input/liste_qui_ne_fonctionne_pas2.csv",sep=";",low_memory=False)
df_liste_non_fonctionne_pas= pd.read_csv("input/liste_non_fonctionne_pas.csv",sep=";",low_memory=False)
df_players=pd.concat([df_players_ligue1, df_players_bundesligua,df_players_premiere_league,df_players_serieA,df_players_liga,df_liste_qui_ne_fonctionne_pas2,df_liste_non_fonctionne_pas], ignore_index=True)
df_players.to_csv('intermed/df_players.csv', sep=";")   

# Dataframe MATCHS

In [ ]:
def test(championnat):
    r = requests.get(championnat)
    soup = BeautifulSoup(r.text, 'html.parser')
    liens=soup.find_all("tbody")[0].find_all("tr")
    list_link=[]
    for i in range(len(liens)):
        lien="https://fbref.com"+liens[i].find("a")["href"]
        list_link.append(lien)

    df_matchs=pd.DataFrame(columns=["date","host_team","away_team","score","host_list_player","away_list_player"])

    for element in range(len(list_link[0:17])): 
        r = requests.get(list_link[element])
        soup = BeautifulSoup(r.text, 'html.parser')
        lien="https://fbref.com"+soup.find_all("ul",{"class":"hoversmooth"})[1].find_all("p")[1].find("a")["href"]
        r2=requests.get(lien)
        soup2 = BeautifulSoup(r2.text, 'html.parser')

        for rapport in range(len(soup2.find_all("tbody")[0].find_all("td",{"data-stat":"score"}))) :
            records=[]    
            if soup2.find_all("tbody")[0].find_all("td",{"data-stat":"score"})[rapport].text!='':
                try: 
                    
                    date=soup2.find_all("tbody")[0].find_all("td",{"data-stat":"date"})[rapport].find("a").text
                    host_team=soup2.find_all("tbody")[0].find_all("td",{"data-stat":"squad_a"})[rapport].find("a").text
                    away_team= soup2.find_all("tbody")[0].find_all("td",{"data-stat":"squad_b"})[rapport].find("a").text
                    score=soup2.find_all("tbody")[0].find_all("td",{"data-stat":"score"})[rapport].find("a").text

                    match="https://fbref.com"+soup2.find_all("tbody")[0].find_all("td",{"data-stat":"match_report"})[rapport].find("a")["href"]
                    r3=requests.get(match)
                    soup3 = BeautifulSoup(r3.text, 'html.parser')
                    host_players=soup3.find_all("tbody")[5].find_all("th",{"scope":"row","data-stat":"player"})
                    host_list_player=[]
                    for i in range(len(host_players)):
                        host_player=host_players[i].find("a").text
                        host_list_player.append(host_player)

                    away_players=soup3.find_all("tbody")[7].find_all("th",{"scope":"row","data-stat":"player"})                
                    away_list_player=[]
                    for i in range(len(away_players)):
                        away_player=away_players[i].find("a").text
                        away_list_player.append(away_player)
                except : 
                    try : 
                        host_players=soup3.find_all("tbody")[0].find_all("th",{"scope":"row","data-stat":"player"})
                        host_list_player=[]
                        for i in range(len(host_players)):
                            host_player=host_players[i].find("a").text
                            host_list_player.append(host_player)

                        away_players=soup3.find_all("tbody")[1].find_all("th",{"scope":"row","data-stat":"player"})                
                        away_list_player=[]
                        for i in range(len(away_players)):
                            away_player=away_players[i].find("a").text
                            away_list_player.append(away_player)
                    except:
                        host_list_player=""
                        away_list_player=""
                        print("problème sur la liste des joueurs : {}".format(match))
                
            else :
                records.append("")
                continue
            records.append((date,host_team,away_team,score,host_list_player,away_list_player))
            df_match = pd.DataFrame(records, columns=["date","host_team","away_team","score","host_list_player","away_list_player"])
            df_matchs=df_matchs.append(df_match)
            sleep(1)
    return df_matchs


In [391]:
serieA="https://fbref.com/fr/comps/11/historique/Saisons-Serie-A"
bundesliga="https://fbref.com/fr/comps/20/historique/Saisons-Bundesliga"
premiere_league="https://fbref.com/fr/comps/9/historique/Saisons-Premier-League"
liga="https://fbref.com/fr/comps/12/historique/Saisons-La-Liga"
ligue1="https://fbref.com/fr/comps/13/historique/Saisons-Ligue-1"

In [ ]:
df_matchs_serieA=test(serieA)
df_matchs_serieA.to_csv('input/df_matchs_serieA.csv',sep=";")

In [ ]:
df_matchs_bundesliga=test(bundesliga)
df_matchs_bundesliga.to_csv('input/df_matchs_bundesliga.csv',sep=";")

In [ ]:
df_matchs_premiere_league=test(premiere_league)
df_matchs_premiere_league.to_csv('input/df_matchs_premiere_league.csv',sep=";")

In [ ]:
df_matchs_liga=test(liga)
df_matchs_liga.to_csv('input/df_matchs_liga.csv',sep=";")

In [ ]:
df_matchs_ligue1=test(ligue1)
df_matchs_ligue1.to_csv('input/df_matchs_ligue1.csv',sep=";")

In [ ]:
df_teams_ligue1= pd.read_csv("input/df_teams_ligue1.csv",sep=";",low_memory=False)
df_teams_liga= pd.read_csv("input/df_teams_liga.csv",sep=";",low_memory=False)
df_teams_premiere_league= pd.read_csv("input/df_teams_premiere_league.csv",sep=";",low_memory=False)
df_teams_budesligua= pd.read_csv("input/df_teams_budesligua.csv",sep=";",low_memory=False)
df_teams_serieA= pd.read_csv("input/df_teams_serieA.csv",sep=";",low_memory=False)
df_teams=pd.concat([df_teams_ligue1, df_teams_liga,df_teams_premiere_league,df_teams_budesligua,df_teams_serieA], ignore_index=True)
df_teams.to_csv('intermed/df_teams.csv', sep=";")   